In [7]:
#!/usr/bin/env python

mydata=dict(
    file1 = 'regex_sum_42.txt',
    file2 = 'regex_sum_267269.txt'
    )

In [86]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
# from numpy import matrix
from scipy import optimize

import math
# from math import pow
from collections import namedtuple
import math
import random
import os
import json

class HdrNeuralNetwork:
    WIDTH_IN_PIXELS = 20
    LEARNING_RATE = 0.1
    # for online learning
    NN_FILE_PATH = 'nn.json'

    def __init__(self, num_hidden_nodes, data_matrix, data_labels, use_file=True):
        self.LAMBDA = 0
        # for regularization of the cost function
        self.num_hidden_nodes = num_hidden_nodes
        self.sigmoid = np.vectorize(self._sigmoid_scalar)
        self.sigmoid_prime = np.vectorize(self._sigmoid_prime_scalar)
        self._use_file = use_file
        self.data_matrix = data_matrix # 2-D list
        self.data_labels = data_labels # 1-D list
        self.sample_num = len(self.data_labels)

        if (not os.path.isfile(HdrNeuralNetwork.NN_FILE_PATH) or not use_file):
            # Step 1: Initialize weights to small numbers
            self.theta1 = self._rand_initialize_weights(self.num_hidden_nodes, 400+1)
            # num_hidden_nodes*401 matrix, the one is the bias
            self.theta2 = self._rand_initialize_weights(10, self.num_hidden_nodes+1)
            # 10*(num_hidden_nodes+1) matrix, the one is the bias

            print 0
            # Train using sample data
            TrainData = namedtuple('TrainData', ['fig', 'label'])
            # print self.theta1.shape
            # print np.mat(self.theta1.flatten(1)).T.shape
            # print np.row_stack((np.mat(self.theta1.flatten(1)).T, np.mat(self.theta2.flatten(1)).T)).shape
            the_temp = tuple(np.row_stack((np.mat(self.theta1.flatten(1)).T, np.mat(self.theta2.flatten(1)).T)).T.tolist()[0])
            theta0 = np.asarray(the_temp) 
            args = tuple([TrainData(self.data_matrix[i], int(self.data_labels[i])) for i in range(self.sample_num)])
            # print type(args)
            # print len(args)
            
            print 1
            res = optimize.fmin_cg(self.nnCostFunction, theta0, fprime=self.nnGrad, args=args)
            res = np.mat(res)
            self.theta1 = np.reshape(res[0:self.num_hidden_nodes*401-1], (400+1, self.num_hidden_nodes)).T
            self.theta2 = np.reshape(res[self.num_hidden_nodes*401:], (10, self.num_hidden_nodes+1)).T
            # self.train([TrainData(self.data_matrix[i], int(self.data_labels[i])) for i in range(self.sample_num)])
            # The parameter is a list of tuples, or a list of dicts, which are called namedtuple
            # print self.theta1.shape
            self.save()
        else:
            self._load()

    def _rand_initialize_weights(self, size_in, size_out):
        return np.random.rand(size_out, size_in)*0.12-0.06

    # The sigmoid activation function. Operates on scalars.
    def _sigmoid_scalar(self, z):
        return 1 / (1 + math.e ** -z)

    def _sigmoid_prime_scalar(self, z):
        return self.sigmoid(z) * (1 - self.sigmoid(z))

    def _draw(self, sample):
        pixelArray = [sample[j:j+self.WIDTH_IN_PIXELS] for j in xrange(0, len(sample), self.WIDTH_IN_PIXELS)]
        plt.imshow(zip(*pixelArray), cmap = cm.Greys_r, interpolation="nearest")
        plt.show()

    def nnCostFunction(self, the_thetas, *args):
        
        # print 3
        the_thetas = np.mat(the_thetas)
        # print the_thetas[0,0:self.num_hidden_nodes*401].shape
        theta1 = np.reshape(the_thetas[0,0:self.num_hidden_nodes*401], (400+1, self.num_hidden_nodes)).T
        theta2 = np.reshape(the_thetas[0,self.num_hidden_nodes*401:], (self.num_hidden_nodes+1, 10)).T
        # print theta2.shape
        training_data_array = args
        
        J=0
        
        for data in training_data_array:
            a1 = np.mat(data.fig).T
            # 400*1 matrix
            # print a1.shape
            z2 = np.dot(theta1, np.row_stack((1, a1)))
            # num_hidden_nodes*1 matrix
            a2 = self.sigmoid(z2)

            z3 = np.dot(theta2, np.row_stack((1, a2)))
            # 10*1 matrix
            a3 = self.sigmoid(z3)

            # Step 3: Back propagation
            y = [0] * 10 # y is a python list for easy initialization and is later turned into an np matrix (2 lines down).
            y[data.label] = 1
            # 1*10 list          
            # print a3.shape
            # print np.mat(y).T.shape
            
            for j in range(10):
                   J = J + np.mat(y).T[j,0]*math.log(a3[j,0])+(1-np.mat(y).T[j,0])*math.log(1-a3[j,0])

        J = -J/self.sample_num + self.LAMBDA/(2*self.sample_num)*(np.multiply(theta1[:,1:], theta1[:,1:]).sum()+np.multiply(theta2[:,1:], theta2[:,1:]).sum())
        
        print J
        return J
        
    def nnGrad(self, the_thetas, *args):
        
        # print 2
        the_thetas = np.mat(the_thetas)
        # print the_thetas[0,0:self.num_hidden_nodes*401].shape
        theta1 = np.reshape(the_thetas[0,0:self.num_hidden_nodes*401], (400+1, self.num_hidden_nodes)).T
        theta2 = np.reshape(the_thetas[0,self.num_hidden_nodes*401:], (self.num_hidden_nodes+1, 10)).T
        # print theta2.shape
        training_data_array = args
        # print type(training_data_array)
        # print type(training_data_array.fig)
        
        Delta1 = np.mat(np.zeros(theta1.shape))
        # print Delta1.shape
        # num_hidden_nodes*401 matrix
        Delta2 = np.mat(np.zeros(theta2.shape))
        # 10*(num_hidden_nodes+1) matrix
        theta1_grad = np.mat(np.zeros(theta1.shape))
        # print theta1_grad.shape
        theta2_grad = np.mat(np.zeros(theta2.shape))
        
        # print 2.1
        # print type(training_data_array)
        for data in training_data_array:
            # Step 2: Forward propagation
            # print len(data.fig)
            a1 = np.mat(data.fig).T
            # 400*1 matrix
            # print a1.shape
            z2 = np.dot(theta1, np.row_stack((1, a1)))
            # num_hidden_nodes*1 matrix
            a2 = self.sigmoid(z2)

            z3 = np.dot(theta2, np.row_stack((1, a2)))
            # 10*1 matrix
            a3 = self.sigmoid(z3)

            # Step 3: Back propagation
            y = [0] * 10 # y is a python list for easy initialization and is later turned into an np matrix (2 lines down).
            y[data.label] = 1
            # 1*10 list
                      
            delta3 = a3 - np.mat(y).T
            # 10*1 matrix
            z2plus = np.row_stack((0, z2))
            # (num_hidden_nodes+1)*1 matrix
            delta2 = np.multiply(np.dot(theta2.T, delta3), self.sigmoid_prime(z2plus))
            # (num_hidden_nodes+1)*1 matrix
            delta2 = delta2[1:,0]
            # num_hidden_nodes*1 matrix
                      
            # Step 4: Sum delta*a.T and calculate the derivatives
            Delta1 = Delta1 + np.dot(delta2, np.row_stack((1, a1)).T)
            Delta2 = Delta2 + np.dot(delta3, np.row_stack((1, a2)).T)
            # print 2.15    
        
        # print 2.3
        # print Delta1[:,0].shape
        # print theta1_grad[:,0].shape
        # print theta1_grad[:,1].shape
        # print type(Delta1[:,0])
        # print type(theta1_grad[:,0])
        theta1_grad[:,0] = Delta1[:,0]/self.sample_num
        theta2_grad[:,0] = Delta2[:,0]/self.sample_num
        theta1_grad[:,1:] = Delta1[:,1:]/self.sample_num + self.LAMBDA/self.sample_num*theta1[:,1:]
        theta2_grad[:,1:] = Delta2[:,1:]/self.sample_num + self.LAMBDA/self.sample_num*theta2[:,1:] 
        
        #print 2.5        
        # print np.row_stack((theta1_grad.flatten(1).T, theta2_grad.flatten(1).T)).shape
        ret = tuple(np.row_stack((theta1_grad.flatten(1).T, theta2_grad.flatten(1).T)).T.tolist()[0])
        return np.asarray(ret)

    def train(self, training_data_array):        
        for data in training_data_array:
            # Step 2: Forward propagation
            a1 = np.mat(data['fig']).T
            # 400*1 matrix
            z2 = np.dot(np.mat(self.theta1), np.column_stack((1, a1)))
            # num_hidden_nodes*1 matrix
            a2 = self.sigmoid(z2)

            z3 = np.dot(np.mat(self.theta2), np.column_stack((1, a2)))
            # 10*1 matrix
            a3 = self.sigmoid(z3)

            # Step 3: Back propagation
            y = [0] * 10 # y is a python list for easy initialization and is later turned into an np matrix (2 lines down).
            y[data['label']] = 1
            # 1*10 matrix
            
            delta3 = a3 - np.mat(y).T
            # 10*1 matrix
            z2plus = np.column_stack((0, z2))
            # (num_hidden_nodes+1)*1 matrix
            delta2 = np.multiply(np.dot(np.mat(self.theta2), delta3), self.sigmoid_prime(z2plus))
            # (num_hidden_nodes+1)*1 matrix
            delta2 = delta2[1:,0]
            # (num_hidden_nodes+1)*1 matrix

            # Step 4: Update weights
            self.theta1 -= self.LEARNING_RATE * np.dot(delta2, a1.T)
            self.theta2 -= self.LEARNING_RATE * np.dot(delta3, a2.T)

    def predict(self, test):
        y1 = np.dot(np.mat(self.theta1), np.mat(test).T)
        y1 =  y1 + np.mat(self.input_layer_bias) # Add the bias
        y1 = self.sigmoid(y1)

        y2 = np.dot(np.array(self.theta2), y1)
        y2 = np.add(y2, self.hidden_layer_bias) # Add the bias
        y2 = self.sigmoid(y2)

        results = y2.T.tolist()[0]
        return results.index(max(results))

    def save(self):
        if not self._use_file:
            return

        json_neural_network = {
            "theta1":[np_mat.tolist()[0] for np_mat in self.theta1],
            "theta2":[np_mat.tolist()[0] for np_mat in self.theta2]
        };
        with open(HdrNeuralNetwork.NN_FILE_PATH,'w') as nnFile:
            json.dump(json_neural_network, nnFile)

    def _load(self):
        if not self._use_file:
            return

        with open(HdrNeuralNetwork.NN_FILE_PATH) as nnFile:
            nn = json.load(nnFile)
        self.theta1 = [np.array(li) for li in nn['theta1']]
        self.theta2 = [np.array(li) for li in nn['theta2']]


In [ ]:
import numpy as np

HIDDEN_NODE_COUNT = 15

data_matrix = np.loadtxt(open('data.csv', 'rb'), delimiter = ',')
data_labels = np.loadtxt(open('dataLabels.csv', 'rb'))

data_matrix = data_matrix.tolist()
# print data_matrix[0:2]
data_labels = data_labels.tolist()
# print data_labels[0:2]

nn = HdrNeuralNetwork(HIDDEN_NODE_COUNT, data_matrix, data_labels)


0
1


/Users/arfu/Documents/workspace/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:45: DeprecationWarning: Non-string object detected for the array ordering. Please pass in 'C', 'F', 'A', or 'K' instead


6.85153419925


In [ ]:
import unittest
import os

datadir=os.path.join(os.path.expanduser('~'),'Documents','workspace','data')
    
class TestMyClass(unittest.TestCase):
    
    hand1=os.path.join(datadir, mydata['file1'])
    hand2=os.path.join(datadir, mydata['file2'])
    #AttributeError: 'dict' object has no attribute 'file2'
    
    def test_sum1(self):
        self.assertEqual(HandleHand(self.hand1).handle(), 445822)
        
    def test_sum2(self):
        self.assertEqual(HandleHand(self.hand2).handle(), 456992)

In [ ]:
suite = unittest.TestLoader().loadTestsFromTestCase(TestMyClass)
unittest.TextTestRunner().run(suite)